In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# AI Platform (Unified) SDK: Train & deploy a TensorFlow model with hosted runtimes (aka pre-built containers)



## Installation

Install the latest (preview) version of AI Platform (Unified) SDK.



In [ ]:
! pip3 install -U google-cloud-aiplatform --user


Install the Google *cloud-storage* library as well.



In [ ]:
! pip3 install google-cloud-storage


### Restart the Kernel

Once you've installed the AI Platform (Unified) SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.



In [ ]:
import os


if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in AI Platform Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.



In [ ]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}


In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)


In [ ]:
! gcloud config set project $PROJECT_ID


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for AI Platform (Unified). We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with AI Platform. Not all regions provide support for all AI Platform services. For the latest support per region, see [Region support for AI Platform (Unified) services](https://cloud.google.com/ai-platform-unified/docs/general/locations)



In [ ]:
REGION = 'us-central1' #@param {type: "string"}


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.



In [ ]:
from datetime import datetime


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")


### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an AI Platform notebook and run the cell, the cell knows to skip executing the authentication steps.*



In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists('/opt/deeplearning/metadata/env_version'):
    if 'google.colab' in sys.modules:
        from google.colab import auth as google_auth
        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.



In [ ]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}


In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.



In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME


Finally, validate access to your Cloud Storage bucket by examining its contents:



In [ ]:
! gsutil ls -al gs://$BUCKET_NAME


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants



#### Import AI Platform (Unified) SDK

Import the AI Platform (Unified) SDK into our Python environment.



In [ ]:
import time


from google.cloud.aiplatform import gapic as aip


from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from google.protobuf.json_format import MessageToJson
from google.protobuf.json_format import ParseDict


#### AI Platform (Unified) constants

Setup up the following constants for AI Platform (Unified):

- `API_ENDPOINT`: The AI Platform (Unified) API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The AI Platform (Unified) API service endpoint for prediction.
- `PARENT`: The AI Platform (Unified) location root path for dataset, model and endpoint resources.



In [ ]:
# API Endpoint
API_ENDPOINT = "{0}-aiplatform.googleapis.com".format(REGION)

# AI Platform (Unified) location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION


## Clients

The AI Platform (Unified) SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (AI Platform).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.



In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(
        client_options=client_options
    )
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(
        client_options=client_options
    )
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(
        client_options=client_options
    )
    return client


def create_job_client():
    client = aip.JobServiceClient(
        client_options=client_options
    )
    return client


clients = {}
clients['model'] = create_model_client()
clients['endpoint'] = create_endpoint_client()
clients['prediction'] = create_prediction_client()
clients['job'] = create_job_client()

for client in clients.items():
    print(client)


## Prepare a trainer script

### Package assembly

In [ ]:
! rm -rf cifar
! mkdir cifar
! touch cifar/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > cifar/setup.cfg

setup_py = "import setuptools\n\
# Requires TensorFlow Datasets\n\
setuptools.setup(\n\
    install_requires=[\n\
        'tensorflow_datasets==1.3.0',\n\
    ],\n\
    packages=setuptools.find_packages())" 
! echo "$setup_py" > cifar/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Custom Training CIFAR-10\n\
Version: 0.0.0\n\
Summary: Demonstration training script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@google.com\n\
License: Public\n\
Description: Demo\n\
Platform: AI Platform (Unified)"
! echo "$pkg_info" > cifar/PKG-INFO

! mkdir cifar/trainer
! touch cifar/trainer/__init__.py


### Task.py contents

In [ ]:
%%writefile cifar/trainer/task.py
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys

tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default='/tmp/saved_model', type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()

def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)


### Store training script on your Cloud Storage bucket

In [ ]:
! rm -f cifar.tar cifar.tar.gz
! tar cvf cifar.tar cifar
! gzip cifar.tar
! gsutil cp cifar.tar.gz gs://$BUCKET_NAME/trainer_cifar.tar.gz
    

## Train a model

### [projects.locations.customJobs.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)

#### Request

In [ ]:
JOB_NAME = "custom_job_TF_" + TIMESTAMP

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest" 
TRAIN_NGPU = 1
TRAIN_GPU = aip.AcceleratorType.NVIDIA_TESLA_K80

worker_pool_specs = [
    {
        "replica_count": 1,
        "machine_spec": {
            "machine_type": 'n1-standard-4',
            "accelerator_type": TRAIN_GPU,
            "accelerator_count": TRAIN_NGPU
        },
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [
                "gs://" + BUCKET_NAME + "/trainer_cifar.tar.gz"
            ],
            "python_module": "trainer.task",
            "args": [
                "--model-dir=" + 'gs://{}/{}'.format(BUCKET_NAME, JOB_NAME),
                "--epochs=" + str(20),
                "--steps=" + str(100),
                "--distribute=" + "single"
            ],
        }
    }
]

training_job = {
    "display_name": JOB_NAME,
    "job_spec": {
        "worker_pool_specs": worker_pool_specs
    }
}

print(
    MessageToJson(
        aip.CreateCustomJobRequest(
            parent=PARENT, 
            custom_job=training_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "customJob": {
    "displayName": "custom_job_TF_20210227173057",
    "jobSpec": {
      "workerPoolSpecs": [
        {
          "machineSpec": {
            "machineType": "n1-standard-4",
            "acceleratorType": "NVIDIA_TESLA_K80",
            "acceleratorCount": 1
          },
          "replicaCount": "1",
          "pythonPackageSpec": {
            "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest",
            "packageUris": [
              "gs://f33-ucaip-trainingaip-20210227173057/trainer_cifar.tar.gz"
            ],
            "pythonModule": "trainer.task",
            "args": [
              "--model-dir=gs://f33-ucaip-trainingaip-20210227173057/custom_job_TF_20210227173057",
              "--epochs=20",
              "--steps=100",
              "--distribute=single"
            ]
          }
        }
      ]
    }
  }
}
```



#### Call

In [ ]:
request = clients["job"].create_custom_job(
    parent=PARENT, 
    custom_job=training_job
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/customJobs/2970106362064797696",
  "displayName": "custom_job_TF_20210227173057",
  "jobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4",
          "acceleratorType": "NVIDIA_TESLA_K80",
          "acceleratorCount": 1
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest",
          "packageUris": [
            "gs://f33-ucaip-trainingaip-20210227173057/trainer_cifar.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://f33-ucaip-trainingaip-20210227173057/custom_job_TF_20210227173057",
            "--epochs=20",
            "--steps=100",
            "--distribute=single"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-27T17:31:04.494716Z",
  "updateTime": "2021-02-27T17:31:04.494716Z"
}
```



In [ ]:
# The full unique ID for the custom training job
custom_training_id = request.name
# The short numeric ID for the custom training job
custom_training_short_id = custom_training_id.split('/')[-1]

print(custom_training_id)


### [projects.locations.customJobs.get](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)

#### Call

In [ ]:
request = clients['job'].get_custom_job(
    name=custom_training_id
)


#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/customJobs/2970106362064797696",
  "displayName": "custom_job_TF_20210227173057",
  "jobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4",
          "acceleratorType": "NVIDIA_TESLA_K80",
          "acceleratorCount": 1
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest",
          "packageUris": [
            "gs://f33-ucaip-trainingaip-20210227173057/trainer_cifar.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://f33-ucaip-trainingaip-20210227173057/custom_job_TF_20210227173057",
            "--epochs=20",
            "--steps=100",
            "--distribute=single"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-27T17:31:04.494716Z",
  "updateTime": "2021-02-27T17:31:04.494716Z"
}
```



In [ ]:
while True:
    response = clients["job"].get_custom_job(name=custom_training_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(20)

# model artifact output directory on Google Cloud Storage  
model_artifact_dir = response.job_spec.worker_pool_specs[0].python_package_spec.args[0].split("=")[-1]
print("artifact location  " + model_artifact_dir)


## Deploy the model

### Load the saved model

In [ ]:
import tensorflow as tf


model = tf.keras.models.load_model(model_artifact_dir)


### Serving function for image data

In [ ]:
CONCRETE_INPUT = "numpy_inputs"

def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(_preprocess, bytes_inputs, dtype=tf.float32, back_prop=False)
    return {CONCRETE_INPUT: decoded_images}  # User needs to make sure the key matches model's input

m_call = tf.function(model.call).get_concrete_function([tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)])

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob

tf.saved_model.save(
    model, 
    model_artifact_dir, 
    signatures={
        'serving_default': serving_fn,
    }
)


### Get the serving function signature

In [ ]:
loaded = tf.saved_model.load(model_artifact_dir)

input_name = list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]

print('Serving function input:', input_name)


*Example output*:
```
Serving function input: bytes_inputs
```



### [projects.locations.models.upload](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models/upload)

#### Request

In [ ]:
model = {
    "display_name": "custom_job_TF" + TIMESTAMP, 
    "metadata_schema_uri": "",
    "artifact_uri": model_artifact_dir,
    "container_spec": {
        "image_uri": "gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest"
    }
}

print(
    MessageToJson(
        aip.UploadModelRequest(
            parent=PARENT, 
            model=model
        ).__dict__["_pb"]
    )
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "custom_job_TF20210227173057",
    "containerSpec": {
      "imageUri": "gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest"
    },
    "artifactUri": "gs://f33-ucaip-trainingaip-20210227173057/custom_job_TF_20210227173057"
  }
}

```



#### Call

In [ ]:
request = clients['model'].upload_model(
    parent=PARENT,
    model=model
)


#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "model": "projects/116273516712/locations/us-central1/models/8844102097923211264"
}
```



In [ ]:
# The full unique ID for the model
model_id = result.model

print(model_id)


## Make batch predictions



### Make the batch input file

Let's now make a batch input file, which you will store in your local Cloud Storage bucket. The batch input file can be JSONL.



In [ ]:
import base64
import cv2
import json
import numpy as np
import tensorflow as tf


(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


test_image_1, test_label_1 = x_test[0], y_test[0]
test_image_2, test_label_2 = x_test[1], y_test[1]

cv2.imwrite('tmp1.jpg', (test_image_1).astype(np.uint8))
cv2.imwrite('tmp2.jpg', (test_image_2).astype(np.uint8))

gcs_input_uri = "gs://" + BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, 'w') as f:
    bytes = tf.io.read_file('tmp1.jpg')
    b64str = base64.b64encode(bytes.numpy()).decode('utf-8')
    f.write(json.dumps({input_name: {'b64': b64str}}) + '\n')
    
    bytes = tf.io.read_file('tmp2.jpg')
    b64str = base64.b64encode(bytes.numpy()).decode('utf-8')
    f.write(json.dumps({input_name: {'b64': b64str}}) + '\n')
    
! gsutil cat $gcs_input_uri


*Example output*:
```
{"bytes_inputs": {"b64": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCAAgACADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD570PxBpmp6nfaEl48lzpUqpewPCU8lpEDqMsOeD26Z55Fa+s3HhnR/Aj6xZjV7rWrW4ke/wBMtLRGRLTaux1cuPnLlhtIAAUEE5490/ao8E6F4b8P3NxZeGksNW1z4h62Iby2t1/eC3ZoozJxwSiKQOhEZJ5JrqZtI8MftFfs56j8YI/hvo/gq1u9C0ywlbTbFoLa+1SOFWlgPGRmNiQzNkiPOflyf1WHFdark0K8UlUbkvJWel1vqmn5n5MuD6MM7qUJzbpxUXazvJSWtmuzTR8iaBoXirx54H1Hxo10mhx2V/8AZltpEE7ByAV8w8YLdRjAHAz1NcSNcXUtev8AwVrE0DajaQ+YZLY4jnXPJXrkjPPTPXGDXvXwi+F3hvwh8Ffip4i1a7GqX7a1b6fp0c84SKO3Wz3FiCdpHnSHDZ2/KAOtfP8A4v8Ah1qOoWul/Efwu4sL+wk8u2IkUi7JRhtwM5RgBkHpz0xXy+F4gzNY6Mqs3NTfvR6a6adj6bGcPZX/AGfKFKEYcqupemurufqP8c9Il/aA8BeHNS+HHh/7Ze634p0rUtMhsFWUJNdsFlR8HAAWWRXBPrmvGvi5+y/B+z1+0ZqHwW+PXx08LaL4VtJI75dOtPEksgfe8krskKIDCZWdCUkyU2MRuVga5X9lr9qAfsk/tCWPjTW9Ol1XwzpurtdXei27gBJTEyJcxBsDcu/OOAwBHBwa8S+JXxltPi3431/x34y8TT/2tqmpy3V1d6h8/mOzFiN46LkgDpgcdOK/HcPxo/qMalONqkn70ei816307I/Xa/C0XjXTrO8EtJdfR/cUfiz4m8aaBJefD/4NXcd4CJ7f/hI7bVXitZ4HkPzSQMvMxRUUTAEqFGCM4EPw/wDAsnhjwZEmrzte6ipKmWeYSbAV+bYTjAJBPTgNjNbOk+HYdL0qPxPcWsN5BK2FaO43q3fHUH8eld34kku/hP4LsvHPiPRtPvZNSkU6fYSFStvED8zsqjLsq5IBwOB1Jri/4iFn2BxSq0Yxulyq8eZLp1f4ms+BMkx2FlRquVm7u0uVvrbRH//Z"}}
{"bytes_inputs": {"b64": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCAAgACADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9qIntrti9vhg3KkLwR69Kbc3FrYskd1LGjOjsqNjJCjLH8Mj8xXw3+yr+3v8ABbUZL2/8L/G/4ja2L0raac/xAvEbTmndtyLFKOd5AwcZwCSccV6X8Xv22/jD4K+L2n+BPA/7H+qeP4v7LSb/AISLQNYjW0ieTmWLfIoUBQiksxA6VxwxtN0VOWn4nTPC1Y1XBHpuqftI6BZ+MrDw/FZSw2dyzRyXl3p8g/eblCgbcjBG/k8dPevU1tCWIKj/AL5r5+8aftTfCqx+H9leeM/i1pXw51aWJvtWkWF1b6ldQnkqnmRqyg9c7fXGag/Zm/aY+HL69d6MPjvr/jVNWm32M19pcgSwREyVZygAJO7PbAFZ08TUjNqpt32/AdSiuVOK2PyC/Zs/4LOfs7/s+fAbQvgz4K/Ywu7rw94Bd4op9WsbfUZ1u5CGlupHBBLSMCd2MYAA4Fe0eGf+Dm/4deO9EuvDvhvSLjSWt7MpPaw+DfNiihYgNvRWK4/hyRjn3r8WvjN8MviF4C+LPiPTvhtZ6lDo8l86W6QswDID0IHUA5x7Ve/ZF1f9pX4C/Gq1+Ifw90PV7e6mgms71o7QP58EowyMrgqwJCnB9K3w+UQxleFF4hw52lzSb5Y3aXM7Juy3dtbHRRzrCu0qlKEl17/fc/W6f/gsjpGtX40z4Zadp1280IVYYPAdsv70nO8ZQnPPToK7z4a/tKftD/ETU7TQPEur6nbpdgMmnrFHak5PUwwquPq3Wvk34QwftUfE/GtfE3xmnhm0LAiy0SwhiupgezSxouzPfb+dfdv7DPwl0rQtcivhZx4Ub1eWQtJu6lmZslmPqfWnmXD+DyjESgsSq1usYyjF+a5tWvkh18+w+IXJQpJeZ//Z"}}
```



### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)



#### Request



In [ ]:
batch_prediction_job = aip.BatchPredictionJob(
    display_name="custom_job_TF" + TIMESTAMP,
    model=model_id,
    input_config={
        "instances_format": "jsonl",
        "gcs_source": {
            "uris": [gcs_input_uri]
        },
    },
    model_parameters=ParseDict(
        {
            'confidenceThreshold': 0.5,
            "maxPredictions": 2
        },
        Value()
    ),
    output_config={
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
        },
    },
    dedicated_resources = {
        "machine_spec": {
            "machine_type": "n1-standard-2",
            "accelerator_type": 0
        },
        "starting_replica_count": 1,
        "max_replica_count": 1
    }
)

print(MessageToJson(
    aip.CreateBatchPredictionJobRequest(
        parent=PARENT,
        batch_prediction_job=batch_prediction_job
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "custom_job_TF_TF20210227173057",
    "model": "projects/116273516712/locations/us-central1/models/8844102097923211264",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://f33-ucaip-trainingaip-20210227173057/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "maxPredictions": 10000.0,
      "confidenceThreshold": 0.5
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://f33-ucaip-trainingaip-20210227173057/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```



#### Call



In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT,
    batch_prediction_job=batch_prediction_job
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/659759753223733248",
  "displayName": "custom_job_TF_TF20210227173057",
  "model": "projects/116273516712/locations/us-central1/models/8844102097923211264",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://f33-ucaip-trainingaip-20210227173057/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "maxPredictions": 10000.0,
    "confidenceThreshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://f33-ucaip-trainingaip-20210227173057/batch_output/"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 1
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-02-27T18:00:30.887438Z",
  "updateTime": "2021-02-27T18:00:30.887438Z"
}
```



In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split('/')[-1]

print(batch_job_id)


### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)



#### Call



In [ ]:
request = clients["job"].get_batch_prediction_job(
    name=batch_job_id
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/659759753223733248",
  "displayName": "custom_job_TF_TF20210227173057",
  "model": "projects/116273516712/locations/us-central1/models/8844102097923211264",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://f33-ucaip-trainingaip-20210227173057/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "confidenceThreshold": 0.5,
    "maxPredictions": 10000.0
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://f33-ucaip-trainingaip-20210227173057/batch_output/"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 1
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_RUNNING",
  "createTime": "2021-02-27T18:00:30.887438Z",
  "startTime": "2021-02-27T18:00:30.938444Z",
  "updateTime": "2021-02-27T18:00:30.938444Z"
}
```



In [ ]:
def get_latest_predictions(gcs_out_dir):
    ''' Get the latest prediction subfolder using the timestamp in the subfolder name'''
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split('/')[-2]
        if subfolder.startswith('prediction-'):
            if subfolder > latest:
                latest = folder[:-1]
    return latest

while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(response.output_config.gcs_destination.output_uri_prefix)
        ! gsutil ls $folder/prediction*

        ! gsutil cat $folder/prediction*
        break
    time.sleep(60)


*Example output*:
```
gs://f33-ucaip-trainingaip-20210227173057/batch_output/prediction-custom_job_TF_TF20210227173057-2021_02_27T10_00_30_820Z/prediction.errors_stats-00000-of-00001
gs://f33-ucaip-trainingaip-20210227173057/batch_output/prediction-custom_job_TF_TF20210227173057-2021_02_27T10_00_30_820Z/prediction.results-00000-of-00001
{"instance": {"bytes_inputs": {"b64": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCAAgACADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD570PxBpmp6nfaEl48lzpUqpewPCU8lpEDqMsOeD26Z55Fa+s3HhnR/Aj6xZjV7rWrW4ke/wBMtLRGRLTaux1cuPnLlhtIAAUEE5490/ao8E6F4b8P3NxZeGksNW1z4h62Iby2t1/eC3ZoozJxwSiKQOhEZJ5JrqZtI8MftFfs56j8YI/hvo/gq1u9C0ywlbTbFoLa+1SOFWlgPGRmNiQzNkiPOflyf1WHFdark0K8UlUbkvJWel1vqmn5n5MuD6MM7qUJzbpxUXazvJSWtmuzTR8iaBoXirx54H1Hxo10mhx2V/8AZltpEE7ByAV8w8YLdRjAHAz1NcSNcXUtev8AwVrE0DajaQ+YZLY4jnXPJXrkjPPTPXGDXvXwi+F3hvwh8Ffip4i1a7GqX7a1b6fp0c84SKO3Wz3FiCdpHnSHDZ2/KAOtfP8A4v8Ah1qOoWul/Efwu4sL+wk8u2IkUi7JRhtwM5RgBkHpz0xXy+F4gzNY6Mqs3NTfvR6a6adj6bGcPZX/AGfKFKEYcqupemurufqP8c9Il/aA8BeHNS+HHh/7Ze634p0rUtMhsFWUJNdsFlR8HAAWWRXBPrmvGvi5+y/B+z1+0ZqHwW+PXx08LaL4VtJI75dOtPEksgfe8krskKIDCZWdCUkyU2MRuVga5X9lr9qAfsk/tCWPjTW9Ol1XwzpurtdXei27gBJTEyJcxBsDcu/OOAwBHBwa8S+JXxltPi3431/x34y8TT/2tqmpy3V1d6h8/mOzFiN46LkgDpgcdOK/HcPxo/qMalONqkn70ei816307I/Xa/C0XjXTrO8EtJdfR/cUfiz4m8aaBJefD/4NXcd4CJ7f/hI7bVXitZ4HkPzSQMvMxRUUTAEqFGCM4EPw/wDAsnhjwZEmrzte6ipKmWeYSbAV+bYTjAJBPTgNjNbOk+HYdL0qPxPcWsN5BK2FaO43q3fHUH8eld34kku/hP4LsvHPiPRtPvZNSkU6fYSFStvED8zsqjLsq5IBwOB1Jri/4iFn2BxSq0Yxulyq8eZLp1f4ms+BMkx2FlRquVm7u0uVvrbRH//Z"}}, "prediction": [0.0407731421, 0.125140116, 0.118551917, 0.100501947, 0.128865793, 0.089787662, 0.157575116, 0.121281914, 0.0312845968, 0.0862377882]}
{"instance": {"bytes_inputs": {"b64": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCAAgACADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9qIntrti9vhg3KkLwR69Kbc3FrYskd1LGjOjsqNjJCjLH8Mj8xXw3+yr+3v8ABbUZL2/8L/G/4ja2L0raac/xAvEbTmndtyLFKOd5AwcZwCSccV6X8Xv22/jD4K+L2n+BPA/7H+qeP4v7LSb/AISLQNYjW0ieTmWLfIoUBQiksxA6VxwxtN0VOWn4nTPC1Y1XBHpuqftI6BZ+MrDw/FZSw2dyzRyXl3p8g/eblCgbcjBG/k8dPevU1tCWIKj/AL5r5+8aftTfCqx+H9leeM/i1pXw51aWJvtWkWF1b6ldQnkqnmRqyg9c7fXGag/Zm/aY+HL69d6MPjvr/jVNWm32M19pcgSwREyVZygAJO7PbAFZ08TUjNqpt32/AdSiuVOK2PyC/Zs/4LOfs7/s+fAbQvgz4K/Ywu7rw94Bd4op9WsbfUZ1u5CGlupHBBLSMCd2MYAA4Fe0eGf+Dm/4deO9EuvDvhvSLjSWt7MpPaw+DfNiihYgNvRWK4/hyRjn3r8WvjN8MviF4C+LPiPTvhtZ6lDo8l86W6QswDID0IHUA5x7Ve/ZF1f9pX4C/Gq1+Ifw90PV7e6mgms71o7QP58EowyMrgqwJCnB9K3w+UQxleFF4hw52lzSb5Y3aXM7Juy3dtbHRRzrCu0qlKEl17/fc/W6f/gsjpGtX40z4Zadp1280IVYYPAdsv70nO8ZQnPPToK7z4a/tKftD/ETU7TQPEur6nbpdgMmnrFHak5PUwwquPq3Wvk34QwftUfE/GtfE3xmnhm0LAiy0SwhiupgezSxouzPfb+dfdv7DPwl0rQtcivhZx4Ub1eWQtJu6lmZslmPqfWnmXD+DyjESgsSq1usYyjF+a5tWvkh18+w+IXJQpJeZ//Z"}}, "prediction": [0.0406896845, 0.125281364, 0.118567884, 0.100639313, 0.12864624, 0.0898737088, 0.157521054, 0.121037535, 0.0313298739, 0.0864133239]}
```



## Make online predictions



### [projects.locations.endpoints.create](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/create)



#### Request



In [ ]:
endpoint = {
    "display_name": "custom_job_TF" + TIMESTAMP
}

print(MessageToJson(
    aip.CreateEndpointRequest(
        parent=PARENT,
        endpoint=endpoint
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "parent": "projects/f33-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "custom_job_TF_TF20210227173057"
  }
}
```



#### Call



In [ ]:
request = clients["endpoint"].create_endpoint(
    parent=PARENT,
    endpoint=endpoint
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/6810814827095654400"
}
```



In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split('/')[-1]

print(endpoint_id)


### [projects.locations.endpoints.deployModel](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)



#### Request



In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "custom_job_TF" + TIMESTAMP,
    "dedicated_resources": {
        "min_replica_count": 1,
        "machine_spec": {
            "machine_type": 'n1-standard-4',
            "accelerator_count": 0
        }
    }
}

print(MessageToJson(
    aip.DeployModelRequest(
        endpoint=endpoint_id,
        deployed_model=deployed_model,
        traffic_split={"0": 100}
    ).__dict__["_pb"])
)


*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/6810814827095654400",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/8844102097923211264",
    "displayName": "custom_job_TF_TF20210227173057",
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-4"
      },
      "minReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}
```



#### Call



In [ ]:
request = clients["endpoint"].deploy_model(
    endpoint=endpoint_id,
    deployed_model=deployed_model,
    traffic_split={"0": 100}
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{
  "deployedModel": {
    "id": "2064302294823862272"
  }
}
```



In [ ]:
# The unique ID for the deployed model
deployed_model_id = result.deployed_model.id

print(deployed_model_id)


### [projects.locations.endpoints.predict](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)



### Prepare file for online prediction

#### Request



In [ ]:
import base64
import cv2
import tensorflow as tf


(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
test_image, test_label = x_test[0], y_test[0]

cv2.imwrite('tmp.jpg', (test_image * 255).astype(np.uint8))
bytes = tf.io.read_file('tmp.jpg')
b64str = base64.b64encode(bytes.numpy()).decode('utf-8')

instances_list = [{'bytes_inputs': {'b64': b64str}}]

prediction_request = aip.PredictRequest(endpoint=endpoint_id)
prediction_request.instances.append(instances_list)

print(MessageToJson(prediction_request.__dict__["_pb"]))


*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/6810814827095654400",
  "instances": [
    [
      {
        "bytes_inputs": {
          "b64": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCAAgACADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD6E1zw/qemaZY669mkdtqsTPZTpMH85Y3KMcKeOR36444NZGj2/ibWPHaaPeHSLXRbq3jSw1O7u3V3u9zb0ZAh+QIFO4EkliCBjnwv9lfxtrviTxBbW974le/0nQ/h5ohms7m4b92bhVlkEfPIDuwJ6gyADgCuWh1fxP8As6/tGad8H5PiRrHjW6tNd1O/iXUr5Z7mx0uSZlinHODiRQCqrgGTGPmwPyqfClGlnM6Em3TSi/N3Wtnto015H6y+MK08kp14QSqScle6tFxel0+6aZ9d6/rvhXwH4407wWtq+uSXth9pa5jcwKUBIbyxzkL0Ock8nHQV2x0NtN0Gw8a6PDOunXc3liO5GZIGxwG6YBxx1x0zkV4L8Xfij4k8X/Gr4V+HdJtDpdgui3GoajJBAXlkuGvNoUEDcD5MYyuN3zEnpX0B4Q+Iunafdap8OPFCG/sL+PzLkGNgbQB1O7Jxh1JOCOvHXNfUYrh/LPqMo0oKDgvdl10117nzGD4izR5hGdWcp8zs4+umisflx8DNXi/Z/wDHviPTfiP4g+x2WieFtV03U5r9miLw2ilonTIySWijZCB6Yr2X4R/tQT/tC/s56f8AGn4C/AvxTrXiq7jksW1G78NxRlNiRxIrzO5EwiVHAePAfeoO1lIrqv2pf2Xz+1t+z3feC9E1GLSvE2paQtraa1cISXiEqu9tKVydrbMZ5Kkg8jIr234a/Bq7+EngjQPAng3wzB/ZOl6ZFa2tpp/yeWiqFB2Hq2ASeuTz15r9ixHBa+vSp1JXpxXuy6vyfpbXuz8jocUyWCVSirTb1j09V95e+E3hnwXr8dn8QPjLaSWZBguP+EcudKSW6gnSMfLHOrcQh2djCSAxY5BxkzfEDx1H4n8ZyvpEC2WnMAwighMe8hvl3gZyQCB15K5xWNq3iKbVNVk8MW91NZzxLllkt9jL2z0I/DrXCeG47T4seNL3wN4c1nULKPTY2GoX8YYNcSkfKisxwis2ASMnk9AK7f8AiHuQ47CulWlKzfM7S5W+vRfgZQ47zvA4qNako3irK8eZLpfVn//Z"
        }
      }
    ]
  ]
}

```



#### Call



In [ ]:
request = clients["prediction"].predict(
    endpoint=endpoint_id,
    instances=instances_list
)


#### Response



In [ ]:
print(MessageToJson(request.__dict__["_pb"]))


*Example output*:
```
{
  "predictions": [
    [
      0.0406113081,
      0.125313938,
      0.118626907,
      0.100714684,
      0.128500372,
      0.0899592042,
      0.157601,
      0.121072263,
      0.0312432405,
      0.0863570943
    ]
  ],
  "deployedModelId": "2064302294823862272"
}
```



### [projects.locations.endpoints.undeployModel](https://cloud.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)



#### Call



In [ ]:
request = clients['endpoint'].undeploy_model(
    endpoint=endpoint_id, 
    deployed_model_id=deployed_model_id, 
    traffic_split={}
)


#### Response



In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))


*Example output*:
```
{}
```



# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.



In [ ]:
delete_model = True
delete_endpoint = True
delete_custom_job = True
delete_batchjob = True
delete_bucket = True

# Delete the model using the AI Platform (Unified) fully qualified identifier for the model
try:
    if delete_model:
        clients['model'].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the AI Platform (Unified) fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients['endpoint'].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the custom training using the AI Platform (Unified) fully qualified identifier for the custom training
try:
    if delete_custom_job:
        clients['job'].delete_custom_job(name=custom_training_id)
except Exception as e:
    print(e)

# Delete the batch job using the AI Platform (Unified) fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients['job'].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME
